In [16]:
import math

In [21]:
class Value:
    def __init__(self, data, _children=()):
        self.data = data
        self.grad = 0.0
        self._prev = set(_children)
        self._backward = lambda: None

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other))

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other))

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,))

        def _backward():
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()

        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)

        build(self)
        self.grad = 1.0

        for v in reversed(topo):
            v._backward()


In [22]:
x1=Value(2.0)
x2=Value(0.0)

w1=Value(-3.0)
w2=Value(1.0)

b= Value(6.64436384)

x1w1=x1*w1
x2w2=x2*w2

x1w1x2w2= x1w1+x2w2
n=x1w1x2w2+b

o = n.tanh()

o.backward()

print(x1.grad, x2.grad)
print(w1.grad, w2.grad)
print(b.grad)


-2.032593081844858 0.6775310272816193
1.3550620545632386 0.0
0.6775310272816193


In [ ]:
import random

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = self.b
        for wi, xi in zip(self.w, x):
            act = act + wi * xi
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

n = Neuron(2)

x = [Value(2.0), Value(0.0)]

o = n(x)
o.backward()

print(o.data)
for p in n.parameters():
    print(p.grad)


-0.961136494418851
0.1524332781924842
0.0
0.0762166390962421
